In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# 设置超参数
d = 20
alpha_u, alpha_v, alpha_w, beta_u, beta_v = 0.01, 0.01, 0.01, 0.01, 0.01
lr = 0.01
epochs = 10

user_num = 943
item_num = 1682
n = 90570

In [3]:
# 读取数据
train_data = pd.read_csv('/Users/chao/workspace/d2l/data/ml-100k/ua.base', sep='\t', names=['userId', 'itemId', 'rating', 'timestamp'])
test_data = pd.read_csv('/Users/chao/workspace/d2l/data/ml-100k/ua.test', sep='\t', names=['userId', 'itemId', 'rating', 'timestamp'])

In [4]:
# 数据预处理
indices = list(range(n))
random.shuffle(indices)
train_data = train_data.iloc[indices,:]

# 生成显示评分矩阵
ratings = np.zeros((user_num + 1, item_num + 1), int)
y_ui = np.zeros((user_num + 1, item_num + 1), int)
for i in range(int(n/2)):
    userId, itemId, rating = train_data.iloc[i,:]['userId'], train_data.iloc[i,:]['itemId'], train_data.iloc[i,:]['rating']
    ratings[userId][itemId] = rating
    y_ui[userId][itemId] = 1
# 生成隐式反馈矩阵
feedbacks = np.zeros((user_num + 1, item_num + 1), int)
for i in range(int(n/2), n):
    userId, itemId, rating = train_data.iloc[i,:]['userId'], train_data.iloc[i,:]['itemId'], train_data.iloc[i,:]['rating']
    feedbacks[userId][itemId] = 1

# 在隐式反馈表中用户u评分过的所有物品
def I_u(u):
    return np.where(feedbacks[u] == 1)[0]

def U_(u, W):
    items = I_u(u)
    w = np.zeros(d, float)
    for item in items:
        w += W[item]
    return (w if len(items) == 0 else w / pow(len(items), 0.5))

In [5]:
# 初始化参数
def init():
    mu = (y_ui * ratings).sum() / y_ui.sum()
    bu = np.zeros(user_num + 1, float)
    for u in range(1, user_num + 1):
        bu[u] = (0 if y_ui[u].sum() == 0 else (y_ui[u] * (ratings[u] - mu)).sum() / y_ui[u].sum())
    bi = np.zeros(item_num + 1, float)
    for i in range(1, item_num + 1):
        bi[i] = (0 if y_ui[:,i].sum() == 0 else (y_ui[:,i] * (ratings[:,i] - mu)).sum() / y_ui[:,i].sum())
    U = np.random.rand(user_num + 1, d)
    V = np.random.rand(item_num + 1, d)
    W = np.random.rand(item_num + 1, d)
    U = (U - 0.5) * 0.01
    V = (V - 0.5) * 0.01
    W = (W - 0.5) * 0.01
    return mu, bu, bi, U, V, W

# 预测函数
def predict_rule(u, i, mu, bu, bi, U, V, W):
    u_ = U_(u, W)
    return U[u] @ V[i].T + u_ @ V[i].T + bu[u] + bi[i]+ mu

In [6]:
mu, bu, bi, U, V, W= init()
train_data = train_data.iloc[0:int(n/2),:]
n = len(train_data)
for epoch in range(epochs):
    for t in range(n):
        # 随机取数据
        u, i, rating = train_data.iloc[t,:]['userId'], train_data.iloc[t,:]['itemId'], train_data.iloc[t,:]['rating']
        # 计算梯度
        e = ratings[u][i] - predict_rule(u, i, mu, bu, bi, U, V, W)
        u_ = U_(u, W)
        delta_mu = -e
        delta_bu = -e + beta_u * bu[u]
        delta_bi = -e + beta_v * bi[i]
        delta_Uu = (-e) * V[i] + alpha_u * U[u]
        delta_Vi = (-e) * (U[u] + u_) + alpha_v * V[i]
        items = I_u(u)
        w = np.zeros(d, float)
        for item in items:
            w += W[item]
        delta_Wi = -e / pow(len(items), 0.5) * V[i] + alpha_w * w
        # 更新参数
        mu -= lr * delta_mu
        bu[u] -= lr * delta_bu
        bi[i] -= lr * delta_bi
        U[u] -= lr * delta_Uu
        V[i] -= lr * delta_Vi
        W[i] -= lr * delta_Wi
    # 学习率下降
    lr *= 0.9

In [7]:
# 预测规则
def SVD_PP(u, j):
    return predict_rule(u, i, mu, bu, bi, U, V, W)

In [8]:
# 损失函数
def MAE(predict_rule):
    data_num = test_data.shape[0]
    loss = 0.0
    for i in range(data_num):
        userId, itemId, rating = test_data.iloc[i,:]['userId'], test_data.iloc[i,:]['itemId'], test_data.iloc[i,:]['rating'] 
        y_hat = postProcess(predict_rule(userId, itemId))
        loss += abs(y_hat - rating)
    return loss / data_num

def RMSE(predict_rule):
    data_num = test_data.shape[0]
    loss = 0.0
    for i in range(data_num):
        userId, itemId, rating = test_data.iloc[i,:]['userId'], test_data.iloc[i,:]['itemId'], test_data.iloc[i,:]['rating'] 
        y_hat = postProcess(predict_rule(userId, itemId))
        loss += ((y_hat - rating) ** 2) / data_num
    return loss ** 0.5

# 数据后处理
def postProcess(num):
    num = min(5.0, num)
    num = max(1.0, num)
    return num

# 预测
def predict(*predict_rules):
    for predict_rule in predict_rules:
        print(f"{RMSE(predict_rule):.4f}, {MAE(predict_rule):.4f}")

In [9]:
# 输出结果
predict(SVD_PP)

1.1602, 0.8902
